In [4]:
from transformers import TFAutoModelForSeq2SeqLM, TFBertForSequenceClassification, TFGPT2LMHeadModel, AutoTokenizer, pipeline
import tensorflow as tf
from flask import Flask, render_template, request, jsonify
import speech_recognition as sr
from cachetools import TTLCache
import os


In [8]:
import torch
print(torch.__version__)

from transformers import pipeline
print("Transformers loaded successfully!")


2.4.1+cpu
Transformers loaded successfully!


In [10]:
cache = TTLCache(maxsize=100, ttl=300)  # Cache 100 responses, expire after 5 minutes


In [12]:
class Chatbot:
    def __init__(self):
        self.translation_tokenizer = AutoTokenizer.from_pretrained('t5-small')
        self.nlu_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
        self.response_tokenizer = AutoTokenizer.from_pretrained('gpt2')
        self.translation_model = None
        self.nlu_model = None
        self.response_model = None

In [14]:
 def load_translation_model(self):
        if self.translation_model is None:
            self.translation_model = TFAutoModelForSeq2SeqLM.from_pretrained('t5-small')
        return self.translation_model

In [16]:
def load_nlu_model(self):
        if self.nlu_model is None:
            self.nlu_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)
        return self.nlu_model

In [18]:
def load_response_model(self):
        if self.response_model is None:
            self.response_model = TFGPT2LMHeadModel.from_pretrained('gpt2')
        return self.response_model

In [52]:
# Import libraries
from transformers import pipeline
import nltk
from googletrans import Translator

# Initialize the Google Translator
translator = Translator()

# Load pre-trained model for text generation
text_generator = pipeline("text-generation", model="gpt2")

# Function to generate chatbot response
def get_response(user_input):
    response = text_generator(user_input, max_length=50, num_return_sequences=1)
    return response[0]['generated_text']

# Function for translation using Google Translate
def translate(text, src_lang="auto", tgt_lang="es"):
    translation = translator.translate(text, src=src_lang, dest=tgt_lang)
    return translation.text

# Multilingual Chatbot Function
def multilingual_chatbot(user_input, user_language="en"):
    # Translate user input to English (if it's not in English)
    if user_language != "en":
        translated_input = translate(user_input, src_lang=user_language, tgt_lang="en")
    else:
        translated_input = user_input
    
    # Get the chatbot response in English
    response = get_response(translated_input)
    
    # Translate the response back to the user's preferred language
    if user_language != "en":
        translated_response = translate(response, src_lang="en", tgt_lang=user_language)
    else:
        translated_response = response
    
    return translated_response

# Example usage
user_input = "¿Cómo estás?"
user_language = "en"  # Spanish

# Get chatbot response in the user's language
response = multilingual_chatbot(user_input, user_language)
print(f"Chatbot response: {response}")


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot response: ¿Cómo estás?


In [54]:
def predict_intent_entities(self, text):
        model = self.load_nlu_model()
        inputs = self.nlu_tokenizer(text, return_tensors="tf", padding=True)
        outputs = model(inputs['input_ids'])
        intent = tf.argmax(outputs.logits, axis=1).numpy()[0]
        entities = []  # Placeholder, entity recognition would require a more complex model or pipeline
        return intent, entities

In [56]:
 def generate_response(self, intent, entities, context):
        model = self.load_response_model()
        prompt = f"Intent: {intent}, Entities: {entities}, Context: {context}\nResponse:"
        inputs = self.response_tokenizer(prompt, return_tensors="tf", padding=True)
        outputs = model.generate(inputs['input_ids'])
        return self.response_tokenizer.decode(outputs[0], skip_special_tokens=True)

In [58]:
 def chatbot_pipeline(self, user_input, user_language):
        cache_key = f"{user_input}-{user_language}"
        if cache_key in cache:
            return cache[cache_key]
        
        translated_input = self.translate(user_input, user_language)
        intent, entities = self.predict_intent_entities(translated_input)
        response = self.generate_response(intent, entities, context={})
        final_response = self.translate(response, 'en', user_language)
        cache[cache_key] = final_response
        return final_response

In [60]:
def chatbot():
    while True:
        user_input = input("You: ")
        if user_input.lower() == "quit":
            print("Bot: Goodbye! Have a great day!")
            break

        intent_tag = predict_intent(user_input)
        
        if intent_tag == "show_info":
            show_info = get_show_info(user_input)
            if show_info:
                response = f"{show_info['show']} is available at {show_info['time']}. Costs: {', '.join([f'{c['type']}: {c['cost']}' for c in show_info['costs']])}"
            else:
                response = "Sorry, I couldn't find that show."
        else:
            response = get_response(intent_tag)
        
        print(f"Bot: {response}")

In [1]:
while True:
    message = input("You: ")
    text = message
    translated = translator.translate(text, src='en', dest='es')
    if message.lower() == "quit":
        break
    response = chatbot_response(message)
    print(f"Bot: {response}")
    print(f"Translated Text: {translated.text}")

You:  helllo


NameError: name 'translator' is not defined

In [32]:
@app.route('/speech-to-text', methods=['POST'])
def speech_to_text():
    recognizer = sr.Recognizer()
    audio_file = request.files['file']

    with sr.AudioFile(audio_file) as source:
        audio = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio)
            return jsonify({"text": text})
        except sr.UnknownValueError:
            return jsonify({"error": "Speech not recognized"}), 400
        except sr.RequestError:
            return jsonify({"error": "Speech recognition service error"}), 500

In [30]:
# Flask app
app = Flask(__name__)

# Initialize chatbot
chatbot = Chatbot()

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/chatbot', methods=['POST'])
def chatbot_response():
    user_input = request.json.get('text')
    user_language = request.json.get('language', 'en')
    response = chatbot.chatbot_pipeline(user_input, user_language)
    return jsonify({"response": response})

In [40]:
if __name__ == '__main__':
    app.run(debug=True, port=5001)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

In [42]:
%tb


SystemExit: 1